# HowTo -Colmap- From RGB images to camera poses

In this notebook you will learn:

1. how to connect to a ROMI database
2. how to use our custom Python wrapper around Colmap to estimate camera poses (extrinsic parameters)
3. how to compare them to the requested CNC camera poses

This notebook **assume** that you have:
- declared the `ROMI_DB` environment variable as the path to the database directory to use
- processed the test dataset with the _geometric pipeline,_ so we can access the fileset containing the data we want to start with...

Remember, the aim of this notebook is to show you how it works "under the hood".
This is not how you should process your data, that is done thanks to the `romi_run_task` CLI tool.

In [ ]:
%matplotlib inline

In [ ]:
import os

import ipywidgets as widgets
from plant3dvision.calibration import pose_estimation_figure
from plant3dvision.colmap import ColmapRunner
from plant3dvision.colmap import colmap_points_to_pcd
from plant3dvision.tasks.colmap import get_image_poses, get_cnc_poses
from plant3dvision.visu import plotly_pointcloud, plotly_image_carousel
from plantdb import FSDB

## Connect to the database & get the initial data

If you did not declare a `ROMI_DB` environment variable, you can do it by uncommenting the next cell and setting it to the right value.

In [ ]:
# os.environ['ROMI_DB'] = "/path/to/test/data"

### Connect to the database

In [ ]:
db = FSDB(os.environ['ROMI_DB'])  # requires definition of this environment variable!
db.connect()

Once you are connected to the database, you can list the available scan *dataset* with `db.list_scans()`.

### Select a dataset

We now select a dataset (with the `Dropdown` widget) for the demo:

In [ ]:
scan_name = widgets.Dropdown(options=db.list_scans(), value=db.list_scans()[0], description='Dataset:')
display(scan_name)

In [ ]:
scan = db.get_scan(scan_name.value)

If you did not process this dataset yet, from the `plant3dvision` root directory, you can do it with:
```
romi_run_task AnglesAndInternodes $ROMI_DB/<selected_dataset> --config plant-3d-vision/configs/geom_pipe_real.toml
```

To list the available *filesets* in this *scan dataset*:

In [ ]:
scan.list_filesets()

### Get the RGB images fileset

The RGB images resulting from a scan by the _Plant Imager_ are to be found in the 'images' fileset.

In [ ]:
img_fs = scan.get_fileset("images")

Once you have access to the 'images' fileset, you may access the RGB images as follows:

In [ ]:
img_files = img_fs.get_files(query={"channel": "rgb"})

In [ ]:
print(f"This fileset contains {len(img_files)} files (matching the `query`).")

### Visualize the RGB images fileset

It is possible to visualize the set of RGB images using our `plotly_image_carousel` method.

In [ ]:
fig = plotly_image_carousel(img_files, title=scan_name.value)

In [ ]:
fig.show()

## Estimate camera poses with Colmap

As stated by the introduction of Colmap's tutorial:
> Image-based 3D reconstruction from images traditionally first recovers a sparse representation of the scene and the camera poses of the input images using Structure-from-Motion.
[Source](https://colmap.github.io/tutorial.html)

This is exactly what we want to achieve here, to _recover the camera poses of the input image_.
These, in combination with masks localizing the plant in the images, will later be used to reconstruct a volume using a *voxel carving* approach.

For now, let's focus on how to obtain these _camera poses_.

In [ ]:
gpu_args = {"feature_extractor": {"--ImageReader.single_camera": "1", "--SiftExtraction.use_gpu": "1"}}

In [ ]:
colmap = ColmapRunner(img_files, matcher_method="exhaustive", all_cli_args=gpu_args)

Note that we use the **exhaustive** matching method.
As these scans have been acquired in sequential order by rotating around the plant, we could use the **sequential** matching method.

However, as we plan to use more complex path than a circular path around the plant, or we could use dataset acquired by other sources, we use the **exhaustive** method by default.

As per the official documentation about the **exhaustive** matching method:
> If the number of images in your dataset is relatively low (up to several hundreds), this matching mode should be fast enough and leads to the best reconstruction results.

As we plan to use fewer images than "several hundreds", we stick with the **exhaustive** matching method.

### Extract image features

As per the official documentation:

> In the first step, feature detection/extraction finds sparse feature points in the image and describes their appearance using a numerical descriptor.

In [ ]:
colmap.feature_extractor()

### Match extracted features from images

As per the official documentation:

> In the second step, feature matching and geometric verification finds correspondences between the feature points in different images.

In [ ]:
colmap.matcher()

### Sparse point cloud reconstruction

As per the official documentation:

> After producing the scene graph in the previous two steps, you can start the incremental reconstruction process

In [ ]:
colmap.mapper()

### Align sparse point cloud to coordinate system of given camera centers

This step is not mandatory, but quite useful to get a similar coordinate orientation as you might expect with the _Plant Imager_.
It aims at aligning the sparse model to coordinate system of given camera centers.

This use the _approximate poses_ given by the _Plant Imager_ (CNC).

In [ ]:
colmap.model_aligner()

### Visualize Colmap sparse point cloud

Once we obtained the sparse point cloud with Colmap, it is time to have a look at it:

In [ ]:
sparse_pcd = colmap_points_to_pcd(f'{colmap.sparse_dir}/0/points3D.bin')
fig = plotly_pointcloud(sparse_pcd)

In [ ]:
fig.show()

### Access the estimated poses

Estimated camera poses (extrinsic parameter) are saved as image metadata:

In [ ]:
img_files[0].get_metadata("estimated_pose")  # XYZ coordinates

Using the `get_image_poses` method from `plant3dvision.tasks.colmap`, you can retrieve all of them at once:

In [ ]:
colmap_poses = get_image_poses(scan, md="estimated_pose")

In [ ]:
colmap_poses['00000_rgb']  # XYZ coordinates

## Compare to expected poses from the _Plant Imager_

It is now time to compare the _estimated poses_ with the _expected poses_, also called _approximate poses_ in the metadata, from the _Plant Imager_.

Indeed, our goal here is to "refine" the _expected poses_ with Colmap as we know that our _Plant Imager_ robot is not perfect, especially the "cheap" motors work in an **open loop**.

### Get approximate poses

Using the `get_cnc_poses` method from `plant3dvision.tasks.colmap`, you can retrieve all of them at once:

In [ ]:
cnc_poses = get_cnc_poses(scan)

### Visualize Colmap estimated poses deviation from approximate poses

You may easily generate a visual comparison of _estimated poses_ versus _approximate poses_ using the `pose_estimation_figure` from `plant3dvision.calibration`.

In [ ]:
fig = pose_estimation_figure(cnc_poses, colmap_poses, pred_scan_id=scan.id, suffix="_estimated")

We may now **disconnect** from the database as we will not need it anymore:

In [ ]:
db.disconnect()